# Documents DB File

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"
file_name = "../data/raw/fintech-scopus.csv"
import_scopus_file(file_name, directory)

# !cat ../data/processed/debug/filter.yaml

# Thesaurus

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"

# clean_institutions(directory)
# clean_keywords(directory)
# apply_thesaurus(directory, thesaurus_file="keywords.txt", input_column="author_keywords", output_column="author_keywords_thesaurus", strict=False)
# create_thesaurus(directory, column="keywords", thesaurus_file="test_thesaurus.txt", sep="; ")

# Basic Analysis

In [1]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"

# core_authors(directory)
# core_sources(directory)
# coverage(directory)
# top_documents(directory)
# summary(directory)


# time_analysis(directory)
# terms_analysis(directory, column='authors')
# impact_analysis(directory, column="authors")
# terms_per_year_analysis(directory, column='countries', metric='num_documents')

# BigraphAnalyzer(dirpath_or_records="../data/processed/debug", column='countries', by='countries').table_

,,countries,united states,china,united kingdom,indonesia,india,australia,south korea,germany,malaysia,italy,...,kuwait,kenya,iraq,iceland,cyprus,cote d'ivoire,cameroon,belize,bangladesh,armenia
,,num_docs,187,186,148,85,78,66,65,61,48,47,...,1,1,1,1,1,1,1,1,1,1
,,cited_by,1614,1159,1081,168,170,531,754,626,118,326,...,18,0,0,1,3,1,46,0,0,1
pub_year,num_docs,cited_by,,,,,,,,,,,,,,,,,,,,,
2015,9,83,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016,28,598,2,4,2,0,0,0,10,2,1,1,...,0,0,0,0,0,0,0,0,0,0
2017,73,1370,9,9,7,1,1,4,8,8,0,3,...,0,0,0,0,0,0,0,0,0,0
2018,132,1801,34,14,15,7,0,7,7,7,4,9,...,0,0,0,0,0,0,0,0,0,1
2019,226,1493,26,25,13,16,18,9,12,12,12,4,...,0,0,1,0,0,0,0,1,0,0
2020,411,1349,56,63,51,40,28,22,18,18,10,18,...,1,0,0,1,1,0,1,0,0,0
2021,415,325,60,70,59,19,30,24,9,13,19,12,...,0,1,0,0,0,1,0,0,1,0


# Plots

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"


# bradford_plot(directory)
# lotka_plot(directory)
# worldmap(directory, metric="num_documents")

In [ ]:
import pandas as pd

pd.read_csv("../data/processed/debug/documents.csv").iso_source_name.value_counts()

In [ ]:
pd.read_csv("../data/processed/debug/documents.csv").columns